# https://blog.naver.com/rising_n_falling/221325080333

In [7]:
import numpy as np
import pandas as pd
from urllib.request import urlopen
from urllib import parse
from urllib.request import Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import json

In [8]:
client_id = "wty59eq5i2"
client_pw = "kGUvQHirsidvpBTbZnYLjk0UdHDjiixtAlJpacNf"

In [23]:
res_type = 'json'

if res_type == 'json':
    api_url = 'https://openapi.naver.com/v1/map/geocode?query='
else:
    api_url = 'https://openapi.nacer.com/v1/map/geocode.xml?query='
    
data = pd.read_excel('cleanhouse.xlsx', usecols='A',names=['도로명주소'])
    
geo_coordi = []
for add in data['도로명주소']:
    add_urlenc = parse.quote(add)
    url = api_url + add_urlenc
    request = Request(url)
    request.add_header('X-Naver-Client-Id',client_id)
    request.add_header('X-Nacer-Client-Secret',client_pw)
    try:
        response = urlopen(request)
    except HTTPError as e:
        print('HTTP Error!')
        latitude = None
        longitude = None
    else:
        rescode = response.getcode()
        if rescode == 200:
            response_body = response.read().decode('utf-8')
            if res_type == 'json':
                response_body = json.loads(response_body)
                if 'result' in responese_body:
                    latitude = response_body['result']['items'][0]['point']['y']
                    longitude = response_body['result']['items'][0]['point']['x']
                    print("Success!")
                else:
                    print("'result' not exist!")
                    latitude = None
                    longitude  = None
            else:
                response_body = BeautifulSoup(response_body, 'lxml')
                latitude = [float(xy.get_text()) for xy in response_body.find_all("y")][0]
                longitude = [float(xy.get_test()) for xy in response_body.find_all("x")][0]
        else:
            print('Response error code : %d' %rescode)
            latitude = None
            longitude = None
            
    geo_coordi.append([latitude, longitude])

HTTP Error!
HTTP Error!
HTTP Error!


SyntaxError: invalid syntax (<ipython-input-22-21bfa5283ca5>, line 10)

In [19]:
np_geo_coordi = np.array(geo_coordi)
pd_geo_coordi = pd.DataFrame({"도로명":data['도로명주소'].values,
                             "위도": np_geo_coordi[:,0],
                             "경도": np_geo_coordi[:,1]})

In [21]:
writer = pd.ExcelWriter('ouput.xlsx')
pd_geo_coordi.to_excel(writer, sheet_name='Sheet1')
writer.save()